In [1]:
from web3 import Web3
from eth_account.signers.local import (
    LocalAccount,
)
from hexbytes import HexBytes
from eth_account.datastructures import (
    SignedTransaction,
)
from web3.contract.contract import (
    Contract,
    ContractFunction
)
from web3.types import (
    TxParams,
    Wei
)
from eth_typing import (
    HexStr,
    HexAddress,
    
)
import time
import json
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
def get_ts() -> int:
    return int(time.time())

Connect To Web3 Wallet

In [3]:
RPC = 'https://mainnet.base.org'
WETH_ADDRESS = HexAddress(HexStr("0x4200000000000000000000000000000000000006"))
private_key = os.getenv('private_key')
w3 = Web3(Web3.HTTPProvider(RPC))
account:LocalAccount = w3.eth.account.from_key(private_key)
wallet_address = Web3.to_checksum_address(account.address)
print(f'wallet_address: {wallet_address}')

wallet_address: 0xbFBE7f5A1C6f3eDFF4c34EbB8211141c57021798


Checking ETH Balance

In [4]:
native_balance = w3.eth.get_balance(wallet_address)/10**18
print(f'native_balance: {native_balance}')

native_balance: 0.004469434535071041


In [5]:
pool_address = Web3.to_checksum_address('0x88A43bbDF9D098eEC7bCEda4e2494615dfD9bB9C')
usdc_address = Web3.to_checksum_address('0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913')
router_address = Web3.to_checksum_address('0x4752ba5dbc23f44d87826276bf6fd6b1c372ad24')

Get Price of Token

In [6]:
pool_abi = json.loads(open(r'abis\UniswapPoolV2.json').read())
erc20_abi = json.loads(open(r'abis\ERC20.json').read())
router_abi = json.loads(open(r'abis\UniswapRouterV2.json').read())

pool_contract:Contract = w3.eth.contract(address=pool_address, abi=pool_abi)
usdc_contract:Contract = w3.eth.contract(address=usdc_address, abi=erc20_abi)
router_contract:Contract = w3.eth.contract(address=router_address, abi=router_abi)

get USDC balance

In [7]:
usdc_balance = pool_contract.functions.balanceOf(wallet_address).call()/10**6
print(f'usdc_balance: {usdc_balance}')

usdc_balance: 0.0


get ETH Price

In [9]:
liquidity = pool_contract.functions.getReserves().call()
price = liquidity[1]/liquidity[0] *10**(18-6)
print(f'ETH Price in USDC: {price}')

ETH Price in USDC: 3469.2255622389944


Swap ETH for USDC

In [14]:
slippage = 0.01 # 1%
eth_amount = int(w3.eth.get_balance(wallet_address)/10)
amount_out = router_contract.functions.getAmountsOut(eth_amount,[WETH_ADDRESS,usdc_address]).call()[1]
min_amount_out = int((1-slippage)*amount_out)
swap_func:ContractFunction = router_contract.functions.swapExactETHForTokens(min_amount_out,[WETH_ADDRESS,usdc_address],wallet_address,get_ts() + 60*10)
nonce = w3.eth.get_transaction_count(wallet_address)
txn_params:TxParams = {'from':wallet_address,
                       'nonce':nonce,
                       'value':Wei(eth_amount),}

txn_params = swap_func.build_transaction(txn_params)
signed_txn_params:SignedTransaction = w3.eth.account.sign_transaction(txn_params,private_key)
txn:HexBytes = w3.eth.send_raw_transaction(signed_txn_params.raw_transaction)

Transaction hash: 48c1af6db88e431ebfe6bc9d3031d904243bd13dbd5987795b3f61a546d0e689
ETH swapped for USDC: 0.00044726818709179 ETH
ETH balance: 0.004024740241840652
USDC balance: 1.551579


In [ ]:
print(f'Waiting for transaction to be mined...')
w3.eth.wait_for_transaction_receipt(txn)
print(f'Transaction hash: {txn.to_0x_hex()}')
print(f'ETH swapped for USDC: {eth_amount/10**18} ETH')
print(f'ETH balance: {w3.eth.get_balance(wallet_address)/10**18}')
print(f'USDC balance: {usdc_contract.functions.balanceOf(wallet_address).call()/10**6}')